# Pre Requisites

In [1]:
# Installing Required Packages
!pip install -q pandasql
# !pip install -q gspread
# !pip install -q gspread-dataframe

In [2]:
# Importing Required Liraries
import pandas as pd
from pandasql import sqldf
import requests
from datetime import datetime, timedelta, timezone
import math

# Functions

In [3]:
# Fetching F1 Data
def fetch_data_to_dataframe(url: str, df_name: str = "data") -> pd.DataFrame:

    print(f"Attempting to fetch {df_name} from: {url}")
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raises an HTTPError for bad responses (4xx or 5xx)
        fetched_data = response.json()

        if fetched_data:
            df = pd.DataFrame(fetched_data)
            print(f"Successfully fetched {len(df)} rows for {df_name}.")
            return df
        else:
            print(f"No data found for {df_name} from the provided URL.")
            return pd.DataFrame()  # Return an empty DataFrame
    except requests.exceptions.RequestException as e:
        print(f"Error fetching {df_name} from {url}: {e}")
        return pd.DataFrame()  # Return an empty DataFrame on any request error

In [10]:
# Fetching F1 Data Without Print Statements
def fetch_data_to_dataframe_wp(url: str, df_name: str = "data", columns_to_omit: list = None) -> pd.DataFrame:

    # print(f"Attempting to fetch {df_name} from: {url}")
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raises an HTTPError for bad responses (4xx or 5xx)
        fetched_data = response.json()

        if fetched_data:
            df = pd.DataFrame(fetched_data)
            # print(f"Successfully fetched {len(df)} rows for {df_name}.")
            if columns_to_omit:
                # Identify columns that actually exist in the DataFrame
                existing_columns_to_drop = [col for col in columns_to_omit if col in df.columns]
                if existing_columns_to_drop:
                    df = df.drop(columns=existing_columns_to_drop)
                    # print(f"Omitted columns: {', '.join(existing_columns_to_drop)}")
                else:
                    print("None of the specified columns to omit were found in the DataFrame.")

            return df
        else:
            print(f"No data found for {df_name} from the provided URL.")
            return pd.DataFrame()  # Return an empty DataFrame
    except requests.exceptions.RequestException as e:
        print(f"Error fetching {df_name} from {url}: {e}")
        return pd.DataFrame()  # Return an empty DataFrame on any request error

In [5]:
# Lap Duration Conversion
def format_lap_duration(seconds):
    if seconds is None or (isinstance(seconds, float) and math.isnan(seconds)):
        return None  # or return 'N/A' if you prefer a string

    minutes, sec = divmod(seconds, 60)
    sec_int = int(sec)
    milliseconds = int(round((sec - sec_int) * 1000))

    return f"{int(minutes)}:{sec_int:02d}.{milliseconds:03d}"

In [32]:
# Function to run SQL queries
pysqldf = lambda q: sqldf(q, globals())

# Required URLs

In [7]:
# Laps and Stints Data For Year
input_year = '2025'
start_date = f"{input_year}-01-01"
end_date   = f"{input_year}-12-31"
## URL to be fetched
sessions_url      = "https://api.openf1.org/v1/sessions"
race_sessions_url = f"https://api.openf1.org/v1/sessions?date_start>{start_date}T00:00:01Z&date_end<{end_date}T00:00:01Z"
meetings_url      = "https://api.openf1.org/v1/meetings"
drivers_url       = "https://api.openf1.org/v1/drivers"
# laps_url          = f"https://api.openf1.org/v1/laps?date_start>{start_date}T00:00:01Z"
result_url        = "https://api.openf1.org/v1/session_result"
grid_url          = "https://api.openf1.org/v1/starting_grid"
# stints_url        = "https://api.openf1.org/v1/stints"

# Data Extraction

In [8]:
# Fetching Data

sessions_df        = fetch_data_to_dataframe(sessions_url, df_name=f"sessions data")
race_sessions_df   = fetch_data_to_dataframe(race_sessions_url, df_name=f"race sessions data")
meetings_df        = fetch_data_to_dataframe(meetings_url, df_name=f"meetings data")
drivers_df         = fetch_data_to_dataframe(drivers_url, df_name="all drivers")
# laps_df            = fetch_data_to_dataframe_wp(laps_url, df_name=f"laps session data",columns_to_omit=columns_to_exclude_sessions)
session_results_df = fetch_data_to_dataframe_wp(result_url, df_name=f"session result data")
starting_grid_df   = fetch_data_to_dataframe_wp(grid_url, df_name=f"starting grid data")
race_sessions      = race_sessions_df[race_sessions_df['session_type'] == 'Race']
race_sessions_all  = sessions_df[sessions_df['session_type'] == 'Race']
# stints             = fetch_data_to_dataframe_wp(stints_url, df_name=f"stints data")

Attempting to fetch sessions data from: https://api.openf1.org/v1/sessions
Successfully fetched 296 rows for sessions data.
Attempting to fetch race sessions data from: https://api.openf1.org/v1/sessions?date_start>2025-01-01T00:00:01Z&date_end<2025-12-31T00:00:01Z
Successfully fetched 60 rows for race sessions data.
Attempting to fetch meetings data from: https://api.openf1.org/v1/meetings
Successfully fetched 61 rows for meetings data.
Attempting to fetch all drivers from: https://api.openf1.org/v1/drivers
Successfully fetched 5918 rows for all drivers.


## Modifying Required Dataframes

In [9]:
# Laps Data

all_laps = []

for idx, row in race_sessions.iterrows():
    session_key = row['session_key']
    laps_url = f"https://api.openf1.org/v1/laps?session_key={session_key}"

    # print(f"Fetching laps for session {session_key}...")
    columns_to_exclude_sessions = ['segments_sector_1', 'segments_sector_2', 'segments_sector_3']
    laps = fetch_data_to_dataframe_wp(laps_url, df_name=f"laps in session {session_key}",columns_to_omit=columns_to_exclude_sessions)

    if laps is not None and not laps.empty:
        laps['session_key'] = session_key  # Optional: keep track of session
        all_laps.append(laps)

# Concatenate all laps into one DataFrame
if all_laps:
    laps_df = pd.concat(all_laps, ignore_index=True)
    print(f"Total laps collected: {len(laps_df)}")
else:
    laps_df = pd.DataFrame()
    print("No laps data found for the selected year.")

# Create a copy of laps_df without the problematic list columns
laps_df['lap_duration_formatted'] = laps_df['lap_duration'].apply(format_lap_duration)


Omitted columns: segments_sector_1, segments_sector_2, segments_sector_3
Omitted columns: segments_sector_1, segments_sector_2, segments_sector_3
Omitted columns: segments_sector_1, segments_sector_2, segments_sector_3
Omitted columns: segments_sector_1, segments_sector_2, segments_sector_3
Omitted columns: segments_sector_1, segments_sector_2, segments_sector_3
Omitted columns: segments_sector_1, segments_sector_2, segments_sector_3
Omitted columns: segments_sector_1, segments_sector_2, segments_sector_3
Omitted columns: segments_sector_1, segments_sector_2, segments_sector_3
Omitted columns: segments_sector_1, segments_sector_2, segments_sector_3
Omitted columns: segments_sector_1, segments_sector_2, segments_sector_3
Omitted columns: segments_sector_1, segments_sector_2, segments_sector_3
Omitted columns: segments_sector_1, segments_sector_2, segments_sector_3
Omitted columns: segments_sector_1, segments_sector_2, segments_sector_3
Total laps collected: 13103


In [11]:
# # Loop through each race session, fetch stints data, and store into stints_df
# all_stints = []

# for idx, row in race_sessions.iterrows():
#     session_key = row['session_key']
#     stints_url = f"https://api.openf1.org/v1/stints?session_key={session_key}"
#     stints = fetch_data_to_dataframe_wp(stints_url, df_name=f"stints in session {session_key}")

#     if stints is not None and not stints.empty:
#         stints['session_key'] = session_key  # Optional: keep track of session
#         all_stints.append(stints)

# # Concatenate all stints into one DataFrame
# if all_stints:
#     stints_df = pd.concat(all_stints, ignore_index=True)
#     print(f"Total stints collected: {len(stints_df)}")
# else:
#     stints_df = pd.DataFrame()
#     print("No stints data found for the selected year.")

In [ ]:
# race_sessions[race_sessions.session_key == 9472]

In [ ]:
# meetings_df[meetings_df.meeting_key == 1229]

In [22]:
laps_df[(laps_df['session_key']==9979)&(laps_df['driver_number']==1)].head()

,meeting_key,session_key,driver_number,lap_number,date_start,duration_sector_1,duration_sector_2,duration_sector_3,i1_speed,i2_speed,is_pit_out_lap,lap_duration,st_speed,lap_duration_formatted
8001,1261,9979,1,1,None,NaN,38.394,21.759,170.0,178.0,False,NaN,258.0,None
8024,1261,9979,1,2,2025-05-25T13:04:36.598000+00:00,34.949,45.145,26.224,83.0,167.0,False,106.318,254.0,1:46.318
8044,1261,9979,1,3,2025-05-25T13:06:22.951000+00:00,32.110,47.430,29.275,159.0,100.0,False,108.815,257.0,1:48.815
8064,1261,9979,1,4,2025-05-25T13:08:11.828000+00:00,31.006,47.889,21.395,136.0,170.0,False,100.290,253.0,1:40.290
8084,1261,9979,1,5,2025-05-25T13:09:52.184000+00:00,21.744,37.485,21.571,177.0,173.0,False,80.800,262.0,1:20.800


In [24]:
session_results_df[(session_results_df.session_key == 9979)&(session_results_df.driver_number == 1)]

,position,driver_number,time_gap,number_of_laps,meeting_key,session_key,points,Q1,Q2,Q3
5519,4,1,20.572,78.0,1261,9979,12.0,NaN,NaN,NaN


In [26]:
# starting_grid_df.head()
starting_grid_df[(starting_grid_df.session_key == 9979)&(starting_grid_df.driver_number == 1)]

,position,driver_number,lap_duration,meeting_key,session_key
1334,4,1,70.669,1261,9979


In [30]:
race_sessions[race_sessions.session_key == 9979]

,meeting_key,session_key,location,date_start,date_end,session_type,session_name,country_key,country_code,country_name,circuit_key,circuit_short_name,gmt_offset,year
42,1261,9979,Monaco,2025-05-25T13:00:00+00:00,2025-05-25T15:00:00+00:00,Race,Race,114,MON,Monaco,22,Monte Carlo,02:00:00,2025


# Joining Data Required for Dashboard

In [73]:
query2 = """
SELECT
    RANK() OVER (PARTITION BY ra.year ORDER BY ra.date_start) as round,
    ROW_NUMBER() OVER (PARTITION BY ra.year ORDER BY ra.date_start) as round_rn,
    md.meeting_name as grand_prix,
    sg.driver_number,
    dr.full_name,
    sg.position as grid_position,
    rs.position as final_position,
    rs.points,
    ra.year



FROM race_sessions_all AS ra
LEFT JOIN meetings_df AS md
  ON md.meeting_key = ra.meeting_key
LEFT JOIN starting_grid_df AS sg
  on sg.session_key = ra.session_key
LEFT JOIN session_results_df AS rs
  ON rs.session_key = ra.session_key
  AND rs.driver_number = sg.driver_number
LEFT JOIN drivers_df AS dr
  ON dr.driver_number = sg.driver_number
  AND dr.session_key = ra.session_key
"""

points = pysqldf(query2)

In [89]:
query3 = """
WITH grand_prix_rounds AS (
    SELECT
        ra.meeting_key,
        ra.year,
        md.meeting_name,
        MIN(ra.date_start) AS grand_prix_date,
        ROW_NUMBER() OVER (PARTITION BY ra.year ORDER BY MIN(ra.date_start)) AS round
    FROM race_sessions_all AS ra
    LEFT JOIN meetings_df AS md
      ON md.meeting_key = ra.meeting_key
    WHERE ra.session_type = 'Race'  -- Only use Race sessions to define the round sequence
    GROUP BY ra.meeting_key, ra.year, md.meeting_name
)

SELECT
    gr.round,
    md.meeting_name AS grand_prix,
    sg.driver_number,
    dr.name_acronym,
    dr.last_name,
    dr.team_name,
    sg.position AS grid_position,
    rs.position AS final_position,
    rs.points,
    ra.year,
    ra.session_name  -- e.g., Race, Sprint, Quali
FROM race_sessions_all AS ra
LEFT JOIN grand_prix_rounds AS gr
  ON gr.meeting_key = ra.meeting_key
  AND gr.year = ra.year
LEFT JOIN meetings_df AS md
  ON md.meeting_key = ra.meeting_key
LEFT JOIN starting_grid_df AS sg
  ON sg.session_key = ra.session_key
LEFT JOIN session_results_df AS rs
  ON rs.session_key = ra.session_key
 AND rs.driver_number = sg.driver_number
LEFT JOIN drivers_df AS dr
  ON dr.driver_number = sg.driver_number
 AND dr.session_key = ra.session_key

"""


points = pysqldf(query3)

In [83]:
# points[(points.grand_prix == 'Chinese Grand Prix')&(points.year == 2025)]

In [17]:
# Laps for each session
query1 = """
SELECT
    md.meeting_name,
    ld.driver_number,
    dr.full_name,
    ld.lap_number,
    ld.lap_duration,
    ld.lap_duration_formatted

FROM race_sessions AS rs
JOIN meetings_df AS md
  ON md.meeting_key = rs.meeting_key
JOIN laps_df AS ld
  ON ld.session_key = rs.session_key
JOIN drivers_df AS dr
  ON dr.driver_number = ld.driver_number
  AND dr.session_key = ld.session_key
WHERE rs.year = 2025
"""

laps_with_sessions = pysqldf(query1)

In [19]:
# laps_with_sessions.head()

In [91]:
# drivers_df[drivers_df.driver_number == 22]

# Extracting Data to Google Sheets

In [58]:
# Connecting Google Sheets Account
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

In [90]:
# Data for Dashboard
spreadsheet_name = 'points_position' # Replace with your sheet's name
worksheet_name = 'Sheet1' # Replace with your desired worksheet name

try:
    sh = gc.open(spreadsheet_name)
    worksheet = sh.worksheet(worksheet_name)
except gspread.exceptions.SpreadsheetNotFound:
    print(f"Spreadsheet '{spreadsheet_name}' not found. Creating a new spreadsheet.")
    sh = gc.create(spreadsheet_name)
    # When you create a new spreadsheet, it automatically has a 'Sheet1'.
    # We'll try to get it, or add if for some reason it's not named 'Sheet1' or we want a different name.
    try:
        worksheet = sh.worksheet(worksheet_name)
    except gspread.exceptions.WorksheetNotFound:
        print(f"Worksheet '{worksheet_name}' not found in new spreadsheet. Creating it.")
        worksheet = sh.add_worksheet(worksheet_name, rows="100", cols="20") # Adjust rows/cols as needed
except gspread.exceptions.WorksheetNotFound:
    print(f"Worksheet '{worksheet_name}' not found in '{spreadsheet_name}'. Creating new worksheet.")
    worksheet = sh.add_worksheet(worksheet_name, rows="100", cols="20") # Adjust rows/cols as needed

# --- ADD THIS LINE TO CLEAR THE SHEET ---
worksheet.clear()
print(f"Worksheet '{worksheet_name}' in '{spreadsheet_name}' cleared.")
# --- END OF ADDITION ---

from gspread_dataframe import set_with_dataframe
set_with_dataframe(worksheet, points, include_index=False, include_column_header=True) # Set include_index=True if you want to export the DataFrame index

print(f"Data exported to: https://docs.google.com/spreadsheets/d/{sh.id}/edit#gid={worksheet.id}")

Worksheet 'Sheet1' in 'points_position' cleared.
Data exported to: https://docs.google.com/spreadsheets/d/12PwGnK8XOKhASXSxS7oD3-kHcWix3kT-f4Pd-MZR1ko/edit#gid=0
